# Career Path Analysis -- 5 Year Plan from 2025 to 2030

## Introduction

As I approach a pivotal point in my career in early 2025, I find myself evaluating potential paths forward. Having spent 3.5 years as an Aerospace Engineer at GE Vernova and recently completing my MS in Aerospace Engineering from Georgia Tech in May 2024, I'm at a natural juncture to consider my next professional moves. This analysis explores different career trajectories over the next five years, from 2025 to 2030, to help inform my decision-making process.

## Simulation Details
This analysis uses a custom Python-based simulation to model different career trajectories and their financial implications over a five-year period from 2025 to 2030.

### Inputs
The simulator can consider any number of different career paths.

For each path, the model accepts a series of annual compensation packages which include
- Base salary
- Performance bonus
- Retirement benefits (employer match)
- RSU grants
- Location-specific tax implications:
  - Federal income tax brackets
  - State income tax
  - Additional local tax where applicable

### Calculations
The simulation performs several key calculations for each year:

1. **Tax Calculations**:
   - Computes effective tax rates using multiple tax schedules (federal, state, and local)
   - Applies progressive tax brackets for accurate tax burden estimation

2. **Compensation Processing**:
   - Calculates after-tax income from multiple sources (salary, bonus, RSUs)
   - Accounts for employer retirement contributions
   - Factors in any one-time adjustments (e.g., grad school payments)

3. **Wealth Accumulation**:
   - Tracks year-over-year wealth changes
   - Maintains running total of net worth
   - Records detailed compensation history

### Expected Results
The simulation will provide:
- Yearly compensation breakdowns by component (salary, bonus, RSUs)
- Effective tax rates by location
- Net worth progression over time
- Total wealth accumulation by the end of 2029

### Assumptions

#### Explicit Assumptions

**Market Conditions:**
   - RSU values remain stable
   - Tax brackets are based on 2025 projections
   - Existing wealth increases by an assumed real market growth rate annually (7% by default)

#### Implicit Assumptions (Limitations)

**The simulation doesn't account for:**
   - Cost of living differences between locations
   - Real estate market implications
   - Potential market volatility affecting RSU values
   - Non-financial factors (career growth, work-life balance)
   - Additional benefits (health insurance, PTO, etc.)
   - Potential changes in tax law or company policies
   - Investment returns on accumulated wealth


In [1]:
# Run the simulation 
import colorsys
import pandas as pd
from config import run_simulation
import plotly.io as pio
pio.renderers.default='iframe'
from plotter import plot_total_wealth_progression, plot_annual_compensation, plot_tax_burden_comparison, plot_comprehensive_comparison

simulation_results = run_simulation()

## The Financials

### Anticipated Career Path Inputs

In [2]:
# Formatting was done by Claude.ai because I've got better things to do than format a DataFrame.
def combine_dfs(df_dict: dict[int, pd.DataFrame]) -> pd.DataFrame:
    index_names = ['Category', 'Year']

    # Concatenate DataFrames
    combined_df = pd.concat(
        df_dict,
        keys=df_dict.keys(),
        names=index_names
    ).reorder_levels([index_names[1], index_names[0]])

    return combined_df

def create_styled_comparison_df(
    df: pd.DataFrame,
    format_dict: dict[str, str],
    caption: str = ""
):
    """
    Creates a styled DataFrame from a dictionary of DataFrames with consistent MultiIndex formatting.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with a multiindex of Category and Year, created by combine_dfs()
    format_dict : dict, optional
        Dictionary of column names to format strings for styling
    caption : str, optional
        Caption for the table. Default is "".
    
    Returns
    -------
    pandas.io.formats.style.Styler
        Styled concatenated DataFrame
    """
    index_names = ['Category', 'Year']
    
    def highlight_categories(df: pd.DataFrame) -> pd.DataFrame:
        """
        Creates a style DataFrame that applies unique background colors to different categories in a MultiIndex DataFrame.
        
        Parameters
        ----------
        df : pd.DataFrame
           Input DataFrame with MultiIndex containing categories to highlight
           
        Returns
        -------
        pd.DataFrame
           DataFrame of same shape as input containing CSS style strings for background colors
           
        Notes
        -----
        Colors are generated using HSV color space with:
        - Evenly spaced hues based on number of categories
        - Fixed saturation of 0.15 for pastel colors
        - Fixed value of 1.0 for full brightness
        
        The function extracts unique categories from the first level of the MultiIndex 
        and assigns each a unique pastel color. Style strings are formatted as 
        'background-color: #RRGGBB' using hex color codes.
        """
        styles = pd.DataFrame('', index=df.index, columns=df.columns)
        categories = df.index.get_level_values(index_names[0]).unique()
        
        n_colors = len(categories)
        colors: dict[str, str] = {}
        for i, category in enumerate(categories):
           hue = i / n_colors
           saturation = 0.15
           value = 1.0
           
           rgb = colorsys.hsv_to_rgb(hue, saturation, value)
           hex_color = '#{:02x}{:02x}{:02x}'.format(
               int(rgb[0] * 255),
               int(rgb[1] * 255),
               int(rgb[2] * 255)
           )
           
           colors[category] = f'background-color: {hex_color}'
        
        for category, color in colors.items():
           styles.loc[(slice(None), category), :] = color
        
        return styles

    
    # Apply styling
    styled_df = (df
        .style
        .format(format_dict)
        .apply(highlight_categories, axis=None)
        .set_properties(**{
            'text-align': 'right',
            'padding': '5px',
            'border': '1px solid #ddd'
        })
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f5f5f5'), 
                                       ('font-weight', 'bold'),
                                       ('text-align', 'left'),
                                       ('padding', '5px'),
                                       ('border', '1px solid #ddd')]},
            {'selector': 'caption', 'props': [('caption-side', 'bottom'), 
                                            ('font-style', 'italic')]}
        ])
        .set_caption(caption)
    )
    
    return styled_df


compensation_df = combine_dfs(simulation_results["compensation_histories"])
compensation_display = create_styled_comparison_df(
    compensation_df,
    format_dict = {
        'Salary': lambda x: f'${x/1000:.0f}K',
        'Bonus': lambda x: f'${x/1000:.0f}K',
        'RSU Grant': lambda x: f'${x/1000:.0f}K',
        'Retirement Match %': '{:.1%}',
        'Effective Tax Rate': '{:.1%}',
        'Income Adjustment': lambda x: f'${x/1000:.0f}K'
    },
    caption='Compensation Comparison Across Career Paths'
)
display(compensation_display)



,,Company,Title,Location,Salary,Bonus,Retirement Match %,RSU Grant,Effective Tax Rate,Income Adjustment,Comment
Year,Category,,,,,,,,,,
2025,GE in SC,GE,PB Engineer,SC,$95K,$5K,7.0%,$80K,26.0%,$0K,None
2026,GE in SC,GE,LPB Engineer,SC,$102K,$10K,7.0%,$0K,23.4%,$0K,None
2027,GE in SC,GE,LPB Engineer,SC,$105K,$10K,7.0%,$80K,26.4%,$0K,None
2028,GE in SC,GE,LPB Engineer,SC,$108K,$11K,7.0%,$0K,23.8%,$0K,None
2029,GE in SC,GE,Senior Engineer,SC,$120K,$18K,7.0%,$0K,24.7%,$0K,None
2025,GE LPB to Google in NYC,GE,LPB Engineer,NY,$102K,$10K,7.0%,$80K,26.0%,$0K,None
2026,GE LPB to Google in NYC,Google,SWE II,NYC,$150K,$22K,7.0%,$33K,30.4%,$-11K,Net Worth Adjustment for Grad School Repayment
2027,GE LPB to Google in NYC,Google,SWE II,NYC,$153K,$8K,7.0%,$33K,29.9%,$0K,None
2028,GE LPB to Google in NYC,Google,SWE III,NYC,$182K,$18K,7.0%,$64K,33.3%,$0K,None


## Simulation Results

In [3]:
worth_df = combine_dfs(simulation_results["worth_histories"])
worth_display = create_styled_comparison_df(
    worth_df,
    format_dict = {
        # Regular numbers
        'Retirement Match %': '{:.1%}',
        'Effective Tax Rate': '{:.1%}',
        
        # Dollar amounts in thousands
        'Gross Salary': lambda x: f'${x/1000:.0f}K',
        'Gross Bonus': lambda x: f'${x/1000:.0f}K',
        'Gross RSUs': lambda x: f'${x/1000:.0f}K',
        'Gross Income': lambda x: f'${x/1000:.0f}K',
        'Net Salary': lambda x: f'${x/1000:.0f}K',
        'Net Bonus': lambda x: f'${x/1000:.0f}K',
        'Net RSUs': lambda x: f'${x/1000:.0f}K',
        'Retirement Match': lambda x: f'${x/1000:.0f}K',
        'Income Adjustment': lambda x: f'${x/1000:.0f}K',
        'Net Income': lambda x: f'${x/1000:.0f}K',
        'Total Tax Burden': lambda x: f'${x/1000:.0f}K',
        'Unrealized Investment Gain': lambda x: f'${x/1000:.0f}K',
        'Total Earned Wealth': lambda x: f'${x/1000:.0f}K',
        
        # Text fields don't need formatting
        # 'Company': None,
        # 'Title': None,
        # 'Location': None,
        # 'Comment': None
    },
    caption='Wealth Progression Across Career Paths'
)
display(worth_display)

,,Company,Title,Location,Comment,Gross Salary,Gross Bonus,Gross RSUs,Retirement Match %,Gross Income,Effective Tax Rate,Total Tax Burden,Net Salary,Net Bonus,Net RSUs,Retirement Match,Income Adjustment,Net Income,Unrealized Investment Gain,Total Earned Wealth
Year,Category,,,,,,,,,,,,,,,,,,,
2025,GE in SC,GE,PB Engineer,SC,None,$95K,$5K,$80K,7.0%,$180K,26.0%,$47K,$70K,$4K,$59K,$7K,$0K,$139K,$11K,$300K
2026,GE in SC,GE,LPB Engineer,SC,None,$102K,$10K,$0K,7.0%,$112K,23.4%,$26K,$78K,$8K,$0K,$7K,$0K,$93K,$21K,$414K
2027,GE in SC,GE,LPB Engineer,SC,None,$105K,$10K,$80K,7.0%,$195K,26.4%,$52K,$77K,$8K,$59K,$7K,$0K,$151K,$29K,$594K
2028,GE in SC,GE,LPB Engineer,SC,None,$108K,$11K,$0K,7.0%,$119K,23.8%,$28K,$82K,$8K,$0K,$8K,$0K,$98K,$42K,$734K
2029,GE in SC,GE,Senior Engineer,SC,None,$120K,$18K,$0K,7.0%,$138K,24.7%,$34K,$90K,$14K,$0K,$8K,$0K,$112K,$51K,$897K
2025,GE LPB to Google in NYC,GE,LPB Engineer,NY,None,$102K,$10K,$80K,7.0%,$192K,26.0%,$50K,$76K,$8K,$59K,$7K,$0K,$149K,$11K,$310K
2026,GE LPB to Google in NYC,Google,SWE II,NYC,Net Worth Adjustment for Grad School Repayment,$150K,$22K,$33K,7.0%,$206K,30.4%,$63K,$104K,$16K,$23K,$11K,$-11K,$142K,$22K,$474K
2027,GE LPB to Google in NYC,Google,SWE II,NYC,None,$153K,$8K,$33K,7.0%,$194K,29.9%,$58K,$107K,$5K,$23K,$11K,$0K,$146K,$33K,$654K
2028,GE LPB to Google in NYC,Google,SWE III,NYC,None,$182K,$18K,$64K,7.0%,$264K,33.3%,$88K,$121K,$12K,$43K,$13K,$0K,$189K,$46K,$888K


In [4]:
# Individual plots
wealth_fig = plot_total_wealth_progression(worth_df)
# tax_fig = plot_tax_burden_comparison(worth_df)

# Comprehensive comparison
# comp_fig = plot_comprehensive_comparison(worth_df)

# Display in notebook
wealth_fig.show()
# tax_fig.show()
# comp_fig.show()

In [5]:
comp_fig = plot_annual_compensation(worth_df)
comp_fig.show()
